# torchmetrics를 통해 모델 평가하기

In [13]:
# !pip install torchmetrics

In [22]:
import torch
from sklearn.metrics import accuracy_score

In [23]:
import numpy as np
preds = torch.randn(10,5).softmax(dim=-1)
preds
preds.argmax(axis=1)

tensor([3, 1, 3, 1, 4, 0, 0, 1, 2, 3])

In [24]:
target = torch.randint(5,(10,))

In [25]:
accuracy_score(preds.argmax(axis=1), target)

0.1

# tensorboard로 시각화하기

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchmetrics.functional import accuracy
from torch.utils.tensorboard import SummaryWriter
import numpy as np

# 1. TensorBoard SummaryWriter 초기화
writer = SummaryWriter('runs/experiment_1')

# 2. 간단한 모델 정의
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 3. 데이터, 모델, 손실 함수, 옵티마이저 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 4. 임의의 학습 데이터 생성
X_train = torch.randn(100, 10).to(device)  # 100개의 샘플, 입력 차원 10
y_train = torch.randint(0, 2, (100,)).to(device)  # 0 또는 1 레이블

# 5. 학습 루프
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Accuracy 계산
    preds = outputs.softmax(dim=-1)
    acc = accuracy(preds, y_train, task="multiclass", num_classes=2)

    # Backward pass 및 최적화
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # TensorBoard에 손실과 정확도 기록
    writer.add_scalar('Loss/train', loss.item(), epoch)
    writer.add_scalar('Accuracy/train', acc.item(), epoch)

    # 진행 상황 출력
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {acc.item():.4f}')

# 6. TensorBoard에 모델 그래프 추가 (선택 사항)
dummy_input = torch.randn(1, 10).to(device)
writer.add_graph(model, dummy_input)

# 7. SummaryWriter 종료
writer.close()

Epoch [20/100], Loss: 0.7256, Accuracy: 0.5100
Epoch [40/100], Loss: 0.7193, Accuracy: 0.4800
Epoch [60/100], Loss: 0.7141, Accuracy: 0.4800
Epoch [80/100], Loss: 0.7097, Accuracy: 0.4900
Epoch [100/100], Loss: 0.7061, Accuracy: 0.4900


In [12]:
model(X_train).to('cpu').detach().numpy().argmax(axis=1)

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

# 검증용 데이터 세트를 통한 평가

In [14]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [15]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

In [16]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [18]:
dataset = CustomDataset("./datasets/non_linear.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

Training Data Size : 160
Validation Data Size : 20
Testing Data Size : 20


In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [20]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1028, -1.7019]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.2816], device='cuda:0', requires_grad=True)], Cost : 0.096
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1013, -1.7025]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4050], device='cuda:0', requires_grad=True)], Cost : 0.078
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1007, -1.7027]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4559], device='cuda:0', requires_grad=True)], Cost : 0.076
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1002, -1.7028]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4769], device='cuda:0', requires_grad=True)], Cost : 0.076
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1002, -1.7030]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4855], device='cuda:0', requires_grad=True)]

In [ ]:
with torch.no_grad(): # 평가용 => grad 수정이 있어선 안되기 때문에 .no_grad 메서드 활용
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)
        print(f"X : {x}")
        print(f"Y : {y}")
        print(f"Outputs : {outputs}")
        print("--------------------")

X : tensor([[27.0400,  5.2000],
        [82.8100,  9.1000],
        [62.4100, -7.9000],
        [ 7.8400, -2.8000]], device='cuda:0')
Y : tensor([[ 75.1200],
        [242.1900],
        [207.5900],
        [ 29.2000]], device='cuda:0')
Outputs : tensor([[ 75.4539],
        [241.6859],
        [207.3993],
        [ 29.5614]], device='cuda:0')
--------------------
X : tensor([[18.4900,  4.3000],
        [ 2.8900, -1.7000],
        [33.6400,  5.8000],
        [ 4.8400, -2.2000]], device='cuda:0')
Y : tensor([[50.0800],
        [12.7900],
        [95.0500],
        [19.0800]], device='cuda:0')
Outputs : tensor([[50.4836],
        [12.3445],
        [94.8905],
        [19.2405]], device='cuda:0')
--------------------
X : tensor([[18.4900, -4.3000],
        [59.2900,  7.7000],
        [94.0900,  9.7000],
        [46.2400,  6.8000]], device='cuda:0')
Y : tensor([[ 65.2300],
        [171.2600],
        [275.3100],
        [132.0400]], device='cuda:0')
Outputs : tensor([[ 65.1280],
        [171

# 모델 불러오기

In [22]:
import torch
from torch import nn


class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# CustomModel 클래스 -> pytorch에서 안전하지 않은 객체로 간주
# model = torch.load('주소', map_location = "cuda" or "cpu")
# weight_only = False
model = torch.load("models/model.pt", map_location=device, weights_only=False)
print(model)

CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)


In [26]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[-0.1281],
        [14.5276],
        [76.9096]], device='cuda:0')


# 모델 전체를 저장하고 불러오는 방법이 아닌, 모델의 상태값만 저장하고 불러옴. 모델의 매개변수만 저장하여 활용

In [27]:
import torch
from torch import nn


class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)

model_state_dict = torch.load("./models/model_state_dict.pt", map_location=device) # 상태 값만 가져오기
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [30]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)

In [31]:
print(outputs)

tensor([[-0.1281],
        [14.5276],
        [76.9096]], device='cuda:0')


# 체크포인트 저장

In [32]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [34]:

train_dataset = CustomDataset("datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [36]:
checkpoint = 1
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        torch.save(
            {
                "model": "CustomModel",
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "cost": cost,
                "description": f"CustomModel 체크포인트-{checkpoint}",
            },
            f"models/checkpoint-{checkpoint}.pt",
        )
        checkpoint += 1

In [37]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.9256],
        [ 69.4991],
        [356.8264]], device='cuda:0')


# 저장한 체크포인트에서 모델 불러오기

In [38]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [39]:
train_dataset = CustomDataset("./datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [41]:
checkpoint = torch.load("./models/checkpoint-6.pt")
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
checkpoint_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]
print(checkpoint_description)

CustomModel 체크포인트-6


In [42]:
print(checkpoint)

{'model': 'CustomModel', 'epoch': 5999, 'model_state_dict': OrderedDict({'layer.weight': tensor([[ 3.0992, -1.7032]], device='cuda:0'), 'layer.bias': tensor([0.5448], device='cuda:0')}), 'optimizer_state_dict': {'state': {}, 'param_groups': [{'lr': 0.0001, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [0, 1]}]}, 'cost': tensor(0.0792, device='cuda:0', requires_grad=True), 'description': 'CustomModel 체크포인트-6'}


In [43]:
for epoch in range(checkpoint_epoch + 1, 10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss
        if (epoch + 1) % 1000 == 0:
            print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 7000, Model : [Parameter containing:
tensor([[ 3.0992, -1.7032]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5405], device='cuda:0', requires_grad=True)], Cost : 0.079
Epoch : 8000, Model : [Parameter containing:
tensor([[ 3.0994, -1.7035]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5365], device='cuda:0', requires_grad=True)], Cost : 0.085
Epoch : 9000, Model : [Parameter containing:
tensor([[ 3.0990, -1.7033]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5329], device='cuda:0', requires_grad=True)], Cost : 0.071
Epoch : 10000, Model : [Parameter containing:
tensor([[ 3.0992, -1.7032]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5295], device='cuda:0', requires_grad=True)], Cost : 0.077


### 텐서플로우랑 다르게 중간 체크포인트를 통해 모델의 학습 내용을 이어서 진행이 가능(텐서플로우도 가능하려나?)

# 이진 분류 예시

In [44]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [45]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([int(self.y[index])])
        return x, y

    def __len__(self):
        return self.length

In [46]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
          nn.Linear(3, 1),
          nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [47]:
dataset = CustomDataset("./datasets/binary.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size], torch.manual_seed(4))
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device) # 이진분류로 BinaryCrossEntropyLoss 활용
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [50]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[0.0035, 0.0004, 0.0043]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.0974], device='cuda:0', requires_grad=True)], Cost : 0.657
Epoch : 2000, Model : [Parameter containing:
tensor([[0.0046, 0.0021, 0.0056]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2223], device='cuda:0', requires_grad=True)], Cost : 0.646
Epoch : 3000, Model : [Parameter containing:
tensor([[0.0041, 0.0016, 0.0048]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.3441], device='cuda:0', requires_grad=True)], Cost : 0.633
Epoch : 4000, Model : [Parameter containing:
tensor([[0.0054, 0.0025, 0.0056]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.4630], device='cuda:0', requires_grad=True)], Cost : 0.622
Epoch : 5000, Model : [Parameter containing:
tensor([[0.0059, 0.0036, 0.0065]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.5791], de

In [51]:
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)

        print(outputs)
        print(outputs >= torch.FloatTensor([0.5]).to(device))
        print("--------------------")

tensor([[0.4688],
        [0.6419],
        [0.3297],
        [0.6899]], device='cuda:0')
tensor([[False],
        [ True],
        [False],
        [ True]], device='cuda:0')
--------------------
tensor([[0.6414],
        [0.5337],
        [0.3580],
        [0.3272]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[0.6265],
        [0.6919],
        [0.7350],
        [0.6624]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
--------------------
tensor([[0.5144],
        [0.3994],
        [0.6239],
        [0.6858]], device='cuda:0')
tensor([[ True],
        [False],
        [ True],
        [ True]], device='cuda:0')
--------------------
tensor([[0.6241],
        [0.7101],
        [0.6372],
        [0.5948]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
--------------------
tensor([[0.5948],
     

# 단층 퍼셉트론 구조

In [52]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.y = df.iloc[:, 2].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [53]:
train_dataset = CustomDataset("./datasets/perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

In [54]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [55]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

C:\Users\601\AppData\Local\Temp\ipykernel_1044\999614697.py:17: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
C:\Users\601\AppData\Local\Temp\ipykernel_1044\999614697.py:18: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.692
Epoch : 2000, Cost : 0.692
Epoch : 3000, Cost : 0.692
Epoch : 4000, Cost : 0.692
Epoch : 5000, Cost : 0.692
Epoch : 6000, Cost : 0.692
Epoch : 7000, Cost : 0.692
Epoch : 8000, Cost : 0.691
Epoch : 9000, Cost : 0.692
Epoch : 10000, Cost : 0.692


In [56]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)
    outputs = model(inputs)
    
    print("---------")
    print(outputs)
    print(outputs <= 0.5)

---------
tensor([[0.4663],
        [0.4983],
        [0.5019],
        [0.5339]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [False]], device='cuda:0')


# 다층 퍼셉트론 구조

In [57]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [58]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.y = df.iloc[:, 2].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(2, 2),
            nn.Sigmoid()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [59]:
train_dataset = CustomDataset("./datasets/perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

In [60]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [61]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

C:\Users\601\AppData\Local\Temp\ipykernel_1044\4096481949.py:10: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
C:\Users\601\AppData\Local\Temp\ipykernel_1044\4096481949.py:11: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.693
Epoch : 2000, Cost : 0.691
Epoch : 3000, Cost : 0.665
Epoch : 4000, Cost : 0.432
Epoch : 5000, Cost : 0.372
Epoch : 6000, Cost : 0.356
Epoch : 7000, Cost : 0.347
Epoch : 8000, Cost : 0.348
Epoch : 9000, Cost : 0.345
Epoch : 10000, Cost : 0.343


In [62]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)
    outputs = model(inputs)
    
    print("---------")
    print(outputs)
    print(outputs <= 0.5)

---------
tensor([[0.0073],
        [0.9925],
        [0.5156],
        [0.5217]], device='cuda:0')
tensor([[ True],
        [False],
        [False],
        [False]], device='cuda:0')
